思路分析：
1、获取单页网页源代码并返回源代码
2、解析单页网页源代码，提取title、actor、time、score数据并存储为生成器
3、将生成器里每一部电影的数据写入txt文档中
4、研究第1-100页url的规律，构建url，调用1、2、3步骤


1、获取单页网页源代码并返回源代码

In [7]:
import requests
import re
import json
from requests.exceptions import RequestException

def get_one_page(url):
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36'}
    response = requests.get(url,headers = headers)
    html = response.text
    return html

In [8]:
print(get_one_page('http://maoyan.com/board/4'))

<!DOCTYPE html>

<!--[if IE 8]><html class="ie8"><![endif]-->
<!--[if IE 9]><html class="ie9"><![endif]-->
<!--[if gt IE 9]><!--><html><!--<![endif]-->
<head>
  <title>TOP100榜 - 猫眼电影 - 一网打尽好电影</title>
  
  <link rel="dns-prefetch" href="//p0.meituan.net"  />
  <link rel="dns-prefetch" href="//p1.meituan.net"  />
  <link rel="dns-prefetch" href="//ms0.meituan.net" />
  <link rel="dns-prefetch" href="//ms1.meituan.net" />
  <link rel="dns-prefetch" href="//analytics.meituan.com" />
  <link rel="dns-prefetch" href="//report.meituan.com" />
  <link rel="dns-prefetch" href="//frep.meituan.com" />

  
  <meta charset="utf-8">
  <meta name="keywords" content="猫眼电影,电影排行榜,热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100">
  <meta name="description" content="猫眼电影热门榜单,包括热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100,多维度为用户进行选片决策">
  <meta http-equiv="cleartype" content="yes" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
  <meta name="renderer" content="webkit" />

  <meta name="HandheldFriendly" content="true" 

2、解析单页网页源代码，提取title、actor、time、score数据并存储为生成器

In [9]:
def parse_one_page(html):
    pattern = re.compile('<dd.*?title="(.*?)".*?star">(.*?)</p>.*?releasetime">(.*?)</p>.*?integer">(.*?)<.*?fraction">(.*?)</i>',re.S)
    movies = re.findall(pattern,html)
    for item in movies:
        yield{
            '电影名':item[0],
            '主演':item[1].strip()[3:],
            '上映时间':item[2][5:],
            '评分':item[3]+item[4]
        }

3、将生成器里每一部电影的数据写入txt文档中

In [10]:
def write_to_txt(content):
    with open('movies.txt','a',encoding='utf8') as f:
        f.write(json.dumps(content,ensure_ascii=False)+'\n')

4、研究第1-100页url的规律，构建url，调用1、2、3步骤

In [11]:
for i in range(0,10):
    url = 'https://maoyan.com/board/4?offset=' + str(i * 10)
    html = get_one_page(url)
    movies= parse_one_page(html)
    for item in movies:
        write_to_txt(item)

# CSV

In [6]:
import requests
import re
import json
import pandas
from requests.exceptions import RequestException

def get_one_page(url):
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36'}
    response = requests.get(url,headers = headers)
    html = response.text
    return html

def parse_one_page(html):
    pageary = []
    pattern = re.compile('<dd.*?title="(.*?)".*?star">'
        + '(.*?)</p>.*?releasetime">(.*?)'
        + '</p>.*?integer">(.*?)<.*?fraction">(.*?)</i>',re.S)
    movies = re.findall(pattern,html)
    for item in movies:
        dict = {
            '电影名':item[0],
            '主演':item[1].strip()[3:],
            '上映时间':item[2][5:],
            '评分':item[3]+item[4]
        }
        pageary.append(dict)
    return pageary

ary = []
for i in range(0,10):
    url = 'https://maoyan.com/board/4?offset=' + str(i * 10)
    html = get_one_page(url)
    pageary = parse_one_page(html)
    ary = ary + pageary
df = pandas.DataFrame(ary)
df.to_csv('movies.csv')